In [36]:
from collections import namedtuple
import os
import urllib.request as request
from zipfile import ZipFile
import shutil
import pybboxes as pbx
import pandas as pd

In [37]:
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/Computer_vision/projects/vehicle_detection_YOLO'

In [38]:
os.path.exists("dataset/dataset.zip")

True

In [39]:
## Change working directory
os.chdir("/Users/vikaslakka/Desktop/FSDS/Computer_vision/projects/vehicle_detection_YOLO")

In [40]:
## Create Dataingestion config
DataIngestionConfig= namedtuple("DataIngestionConfig",
                                [
                                    "root_dir",
                                    "source_url",
                                    "local_data_file",
                                    "unzip_dir",
                                    "zip_file_dir"
                                    "dataset_source"

                                ])

In [41]:
## Create data classes to consider from data ingestion Configs
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    zip_file_dir: Path
    dataset_source: Path

#### Unzipping the dataset available and storing to right folder

In [7]:
from vehicle_detection.constants import *
from vehicle_detection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        """
        Creating artifact directories from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Creating Directories
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Creating directory for data ingestion from config.yaml file.

        Returns:
            DataIngestionConfig: Dataingestion data
        """
        config= self.config.data_ingestion
        ##Create directory that are required for data ingestion(taken from config.yaml)
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir,
            zip_file_dir= config.zip_file_dir,
            dataset_source= config.dataset_source
        )
        return data_ingestion_config



### now create data set and download into respective folders

In [9]:
## Data Ingestion component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
    def _preprocess(self, zf: ZipFile, f:str, working_dir:str):
        """places all files into target folder
           ignores files which has size of zero

        Args:
            zf (ZipFile): Zip file
            f (str): file name with path
            working_dir (str): targer working directory
        """
        target_filepath= os.path.join(working_dir, f)
        dir_name= os.path.dirname(target_filepath)
        #print(f"filepath: {f}")
        
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)##Create file folder
        print(f"target file path: {target_filepath}")
        
        
        
        ##If the folder has size of zero then delete.
        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)
    def copy_dataset_destination(self, source: Path, destination: Path):
        ###Copy dataset into Data ingestion folder
        shutil.copy2(source, destination)
        
    def unzip_and_clean(self):
        """unzip the dataset and clean the files 
           which has zero size
        """
        ##print(self.config.local_data_file)
        with ZipFile(file= self.config.local_data_file, mode="r") as zf:
            ##Getting the list of files available.
            list_of_files= zf.namelist()
            zf.extractall(self.config.unzip_dir)

            ##extracting files that has only filename dataset
            '''for f in list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)'''
    
    def boundingbox_conversion_YOLO(self):
        """conversion of normal co-ordinates 
            to coordinates that yolo understands"""
        config= self.config
        ## Read the coordinates for conversion
        dataresult= pd.read_csv('artifacts/data_ingestion/dataset/train_solution_bounding_boxes (1).csv')
        with open(config.coordinates_path, 'r') as dataresult:
            ##Getting coordinates of each pic and convert
            for coordinates in dataresult[1:]:
                ## Text to list conversion
                coordinates= coordinates.split(',')
                ## Looping around co-ordinates for conversion
                image_name= coordinates[0]
                object_code=0## default to 0 as there is only one detector
                object_coordinates= coordinates[1:]
                ##Creating new text file with image file name and adding the coordinates
                image_path= os.path.join(config.coordinates_target_path, image_name.replace('.jpg', '.txt'))
                ## If the file exists then append(when there are multiple detections) the coordinates else create new file
                if os.path.exists(image_path):
                    write_mode='a'
                else:
                    write_mode='w'
                ## IF there are multiple coordinates in an image then append
                ## 0 means car in this case. If there are multiple detectors then the number changes accordingly
                if os.path.exists(image_path):
                    classification_code= '\n0 '
                else:
                    classification_code='0 '
                ##Write the coordinates into file
                with open(image_path, write_mode) as imagefile:
                    ##Conversion of coordinates from string to number
                    ## Last coordinates has \n so trimming it using rstrip
                    str_to_val_coordinates= [float(num.rstrip('\n')) for num in object_coordinates]
                    yolo_coordinates= pbx.convert_bbox(str_to_val_coordinates, from_type='voc', to_type='yolo', image_size= config.yolo_conversion_image_size)
                    imagefile.write(classification_code)
                    imagefile.write(' '.join([str(num) for num in yolo_coordinates]))

### Coversion of co-ordinates to yolo format

### Try out Data ingestion

In [ ]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.copy_dataset_destination(data_ingestion_config.dataset_source, data_ingestion_config.unzip_dir)
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e 

[2023-10-04 09:05:18,002: INFO: common]: yaml file:configs/config.yaml loaded successfully
[2023-10-04 09:05:18,004: INFO: common]: yaml file:params.yaml loaded successfully
[2023-10-04 09:05:18,005: INFO: common]: Created directory at:artifacts
[2023-10-04 09:05:18,007: INFO: common]: Created directory at:artifacts/data_ingestion


In [7]:
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/Computer_vision/projects/vehicle_detection_YOLO'

In [8]:
import pandas as pd
df= pd.read_csv('artifacts/data_ingestion/dataset/train_solution_bounding_boxes (1).csv')

In [31]:
len(df)

559

In [47]:
dataresult.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [48]:
dataresult['image'][2]

'vid_4_10040.jpg'

In [ ]:
dataresult= pd.read_csv('artifacts/data_ingestion/dataset/train_solution_bounding_boxes (1).csv')

## Loop around data frame
for row in range(len(dataresult)):
    image_name=dataresult['image'][row]
    object_code=0## default to 0 as there is only one detector
    object_coordinates= [dataresult['xmin'][row], dataresult['ymin'][row],
                          dataresult['xmax'][row], dataresult['ymax'][row] ]
    

In [58]:
list(dataresult.columns)
[dataresult['xmin'][2], dataresult['xmax'][2]]

[239.1924747, 361.9681621]

In [ ]:

## Read the coordinates for conversion
with open(config.coordinates_path, 'r') as dataresult:
    ##Getting coordinates of each pic and convert
    for coordinates in dataresult[1:]:
        ## Text to list conversion
        coordinates= coordinates.split(',')
        ## Looping around co-ordinates for conversion
        image_name= coordinates[0]
        object_code=0## default to 0 as there is only one detector
        object_coordinates= coordinates[1:]
        ##Creating new text file with image file name and adding the coordinates
        image_path= os.path.join(config.coordinates_target_path, image_name.replace('.jpg', '.txt'))
        ## If the file exists then append(when there are multiple detections) the coordinates else create new file
        if os.path.exists(image_path):
            write_mode='a'
        else:
            write_mode='w'
        ## IF there are multiple coordinates in an image then append
        ## 0 means car in this case. If there are multiple detectors then the number changes accordingly
        if os.path.exists(image_path):
            classification_code= '\n0 '
        else:
            classification_code='0 '
        ##Write the coordinates into file
        with open(image_path, write_mode) as imagefile:
            ##Conversion of coordinates from string to number
            ## Last coordinates has \n so trimming it using rstrip
            str_to_val_coordinates= [float(num.rstrip('\n')) for num in object_coordinates]
            yolo_coordinates= pbx.convert_bbox(str_to_val_coordinates, from_type='voc', to_type='yolo', image_size= config.yolo_conversion_image_size)
            imagefile.write(classification_code)
            imagefile.write(' '.join([str(num) for num in yolo_coordinates]))